## carport 

snakefood3 pydeps import-deps

In [2]:
#| export
import ast 
from pathlib import Path 
from IPython.display import SVG

def module2path(mod_name, root='.'):
    return root/Path(*mod_name.split('.')).with_suffix('.py')

def path2module(path, root='.'):
    def str2abs(p): return Path(p).absolute()
    rel_path = str2abs(path).relative_to(str2abs(root))
    return '.'.join(rel_path.with_suffix("").parts)

def psplit(path, sep='.'):
    p = path.split(sep)
    return [sep.join(p[:i+1]) for i, _ in enumerate(p)]

# TODO: imports in __inin__.py
def del_init(mod_name):
    if mod_name.endswith(".__init__") :
        return mod_name.replace('.__init__', '') 
    return mod_name 
    
def add_init(mod_name):
    if mod_name.endswith(".__init__"):
        return mod_name 
    return f"{mod_name}.__init__"

def imps_from_file(path, root='.'):
    code = path.read_text(encoding="utf8")
    mod_ast = ast.parse(source=code, filename=path)
    imps = set()
    for node in ast.walk(mod_ast):
        # parse imports
        if isinstance(node, ast.Import):
            for n_ast in node.names:
                imported = (None, n_ast.name, n_ast.asname, None)
        elif isinstance(node, ast.ImportFrom):
            # display(ast.dump(node))
            for n_ast in node.names:
                imported = (node.module, n_ast.name, n_ast.asname, node.level)
        else:
            continue   
        # translate (where, imported_name, as_name, level)
        where, imported_name, _, level = imported
        if level is None:
            imp = imported_name
        elif level == 0:
            imp = f"{where}.{imported_name}"
        else: 
            pa_mod = path2module(Path(*path.parts[:-level]), root)
            imp = f"{pa_mod}.{where}.{imported_name}"
        imps.add(imp)
    return imps

def get_imps(
    root:str='.', # directory where to look for import structure
    project:str='', # name of concerned project module 
    agg_external:bool = True, # whether to aggregate external imports by subsuming descendant modules
    agg_internal:bool = True  # whether to aggregate internal imports by subsuming non-modular leaves
    ):
    mod2path = {}
    mod2imp_in, mod2imp_ex = {}, {}
    mod2imppath_in = {}  # todo: mod2imppath_ex 
    root = Path(root).absolute()
    for path in (root/project).rglob("*.py"):
        mod = path2module(path, root)
        imps = imps_from_file(path, root) 
        mod2path[mod] = path
        for imp in imps: 
            imp_root = imp.split('.')[0]  # snake root 
            if imp_root == project: # i.e., imported module is an internal module
                if agg_internal:
                    imp_path = module2path(imp, root)
                    imp = imp if imp_path.exists() else imp.rsplit('.', 1)[0] 
                mod2imp_in.setdefault(mod, set()).add(imp)
                mod2imppath_in.setdefault(mod, set()).add(imp_path)
            else:
                imp = imp_root if agg_external else imp
                mod2imp_ex.setdefault(mod, set()).add(imp) 
    return dict(
        mod2path=mod2path,
        mod2imp_in=mod2imp_in, 
        mod2imp_ex=mod2imp_ex, 
        mod2imppath_in=mod2imppath_in,
    )

class ImportGraph:
    def __init__(self, root:str='.', project:str='', **kwargs):
        for k, v in get_imps(root, project, **kwargs).items():
            setattr(self, k, v)
        self._edges = [
            *[(fr, to, {'type': 'external'}) for to, frs in self.mod2imp_in.items() for fr in frs],
            *[(fr, to, {'type': 'internal'}) for to, frs in self.mod2imp_ex.items() for fr in frs],
            ]
        self.edges = [(fr, to) for fr, to, data in self._edges]
        self.nodes = sorted(set().union(*self.edges))
    
    def to_nx(self, ignore_nodes=[], **kw):
        import networkx as nx
        
        g = nx.MultiDiGraph(self.edges)
        g.remove_nodes_from(ignore_nodes)
        return g
    
    def to_dot(self, path='', **kw):
        # https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.nx_pydot
        import networkx as nx
        from io import StringIO
        
        string_io = StringIO() 
        g = self.to_nx(**kw)
        nx.nx_pydot.write_dot(g, path or string_io) 
        return string_io.getvalue()
    
    def to_d2(self, ignore_nodes=[], **kw):
        # TODO: `py_d2` may be the better way to do this.`
        # https://github.com/MrBlenny/py-d2
        e_string = '\n'.join({f"{fr} -> {to}" for fr, to in self.edges})
        
        deletes_hooks = '\n'.join(f"{i}: null" for i in ignore_nodes) if ignore_nodes else ""
        options = '''
        vars: { 
            d2-config: { 
                layout-engine: elk 
                theme-id: 200
                } 
            }
        direction: right
        **.style.border-radius: 99
        '''
        d2_string = f"{options}\n{e_string}\n{deletes_hooks}"
        return  d2_string

    def draw_dot(self, **kw):
        import graphviz as gv  
        gv_g = gv.Digraph()
        gv_g.body += str(gv.Source(self.to_dot(**kw))).splitlines()[1:-1]  # read dot string
        gv_g.attr(rankdir='LR')
        return gv_g
    
    def draw_d2(self, **kw):
        d2_string = self.to_d2(**kw) 
        return D2API.create_svg(d2_string)
    
# class PackageImportGraph(ImportGraph):
#     pass
    

class D2API:
    @staticmethod
    def create_svg(d2:str, app:str='d2') -> SVG:
        if app == 'd2':
            import subprocess
            d2_fname = "_temp.d2"
            with open(d2_fname, 'w', encoding='utf-8') as f:
                f.write(d2)
            output = subprocess.getoutput(f'd2 {d2_fname} - ')
            output = output.rsplit('\n',1)[0]
            res = SVG(output)
        elif app == 'kroki':
            from kroki import diagram_image
            # slow
            res = diagram_image(d2, diagram_type='d2', output_format='svg')
        return res

        
if __name__ == "__main__":
    
    project='certx'
    root = 'd:/Studies/Collections/Projects/certx-dev'
    depg = ImportGraph(root, project)
    # ignore_nodes = ["*.utils", 're', 'io', 'itertools', 'argparse', 'IPython', 'pprint']
    # dotg = depg.draw_dot(ignore_nodes=['re', 'io', 'itertools', 'argparse', 'IPython', 'pprint'])
    # d2g = depg.draw_d2(ignore_nodes=["*.utils", 're', 'io', 'itertools', 'argparse', 'IPython', 'pprint'])


## snakefood3

In [ ]:
from snakefood3.gen_deps import GenerateDependency

project_name='certx'
root = 'd:/Studies/Collections/Projects/certx'

dot = GenerateDependency(root, project_name).make_dot_file()
dot = "\n".join((dot).split("\n")[3:]).replace('dpi="150",', '')


In [ ]:
import graphviz as gv  

gv_g = gv.Digraph()
gv_g.body += str(gv.Source(dot)).splitlines()[1:-1]  # read dot string
gv_g
